# Fine-tuning

Test several pre-trained models from HuggingFace, fine tune on adus data, and hypertune parameters

In [1]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install "ray[tune]" optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Preparing our Data

In this notebook, we'll start by using a local dataset (instead of using a dataset stored at Hugging Face).
Let's load data for our classification task.

### Loading dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

# Importing the dataset
dataset = pd.read_excel('/content/drive/Shareddrives/PLN/Assignment 2/data/OpArticles_ADUs.xlsx')
dataset = dataset.drop(columns=['article_id', 'annotator', 'node','ranges'])
dataset['label'].replace(['Value', 'Value(+)', 'Value(-)', 'Fact', 'Policy'],[0,1,2,3,4], inplace=True)

print(dataset.info())
print(dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16743 entries, 0 to 16742
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  16743 non-null  object
 1   label   16743 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 261.7+ KB
None
                                              tokens  label
0           O facto não é apenas fruto da ignorância      0
1  havia no seu humor mais jornalismo (mais inves...      0
2                              É tudo cómico na FIFA      0
3  o que todos nós permitimos que esta organizaçã...      0
4            não nos fazem rir à custa dos poderosos      0


For ease of usage with Transformer models, we convert the dataset into a Hugging Face dataset and split it into train, validation and test sets.

In [4]:
from datasets import Dataset

dataset_hf = Dataset.from_pandas(dataset)

In [5]:
from datasets import DatasetDict

# 90% train, 10% test+validation
train_test = dataset_hf.train_test_split(test_size=0.1, shuffle=True, seed=42)

# Split the 10% test+validation set in half test, half validation
valid_test = train_test['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

# gather everyone if you want to have a single DatasetDict
train_valid_test_dataset = DatasetDict({
    'train': train_test['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

In [6]:
train_valid_test_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'label'],
        num_rows: 15068
    })
    validation: Dataset({
        features: ['tokens', 'label'],
        num_rows: 837
    })
    test: Dataset({
        features: ['tokens', 'label'],
        num_rows: 838
    })
})

## Fine-tuning a pretrained model

### Tokenizer

We first load the tokenizer for our model:

In [7]:
from transformers import AutoTokenizer

def get_tokenizer(name):
    return AutoTokenizer.from_pretrained(name)

Now we need to [preprocess](https://huggingface.co/docs/transformers/preprocessing) our data.

Obtaining the length of the longest sequences in our data splits

In [8]:
def find_max_length(dataset):
    return len(max(dataset, key=lambda x: len(x.split())).split())

train_max_length = find_max_length(train_valid_test_dataset["train"]["tokens"])
val_max_length = find_max_length(train_valid_test_dataset["validation"]["tokens"])
test_max_length = find_max_length(train_valid_test_dataset["test"]["tokens"])

print(f"Longest sequence in train set has {train_max_length} words")
print(f"Longest sequence in val set has {val_max_length} words")
print(f"Longest sequence in test set has {test_max_length} words")

Longest sequence in train set has 81 words
Longest sequence in val set has 51 words
Longest sequence in test set has 59 words


Tokenize entire dataset

In [9]:
# Define tokenizer later
tokenizer = None

def tokenize_dataset(sample):
    return tokenizer(sample["tokens"], truncation=True, max_length=81, padding="max_length")

def get_tokenized_data(dataset):
    return dataset.map(tokenize_dataset, batched=True, remove_columns=["tokens"])

### Loading the model

Since we want to use the model for classification, we should load it with an appropriate classification head:

In [10]:
from transformers import AutoModelForSequenceClassification

def get_model(model_name):
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)
    model.cuda()

    return model

### Fine-tuning

The next step is to [fine-tune](https://huggingface.co/docs/transformers/training) the model with our train data. To do so, we can make use of a [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer).
There are several aspects of training that you can specify via [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).

In [11]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from IPython.display import display

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def get_trainingArgs():
    return TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        data_seed=42,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1"
    )

def get_trainer(model_, args_, dataset_, tokenizer_, data_collator_, compute_metrics_):
    return Trainer(
        model=model_,
        args=args_,
        train_dataset=dataset_["train"],
        eval_dataset=dataset_["validation"],
        tokenizer=tokenizer_,
        data_collator=data_collator_,
        compute_metrics=compute_metrics_
    )

def train_model(model_name):
  global tokenizer
  tokenizer = get_tokenizer(model_name)
  tokenized_dataset = get_tokenized_data(train_valid_test_dataset)
  model = get_model(model_name)

  trainer = get_trainer(
    model,
    get_trainingArgs(),
    tokenized_dataset,
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  # Train Model
  display(trainer.train())

  # Check performance in validation set
  display(trainer.evaluate())

  # Check how the model fares in our test set.
  display(trainer.predict(test_dataset=tokenized_dataset["test"]))

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/baseline/' + model_name)


#### Custom training to use a weighted loss

Useful for our unbalanced training set

In [12]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_valid_test_dataset['train']['label']),
    y=train_valid_test_dataset['train']['label']
    )
class_weights

array([0.41220079, 2.36732129, 1.15066819, 0.92356727, 5.00598007])

In [13]:
from torch import nn, tensor

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        #inputs = inputs.to(device)
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (5 labels with different weight)
        loss_fct = nn.CrossEntropyLoss(weight=tensor([0.41, 2.37, 1.15, 0.92, 5.01]))
        loss_fct.cuda()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def get_trainer_custom(model_, args_, dataset_, tokenizer_, data_collator_, compute_metrics_):
    return CustomTrainer(
        model=model_,
        args=args_,
        train_dataset=dataset_["train"],
        eval_dataset=dataset_["validation"],
        tokenizer=tokenizer_,
        data_collator=data_collator_,
        compute_metrics=compute_metrics_
    )

def train_model_custom(model_name):
  global tokenizer
  tokenizer = get_tokenizer(model_name)
  tokenized_dataset = get_tokenized_data(train_valid_test_dataset)
  model = get_model(model_name)

  trainer = get_trainer_custom(
    model,
    get_trainingArgs(),
    tokenized_dataset,
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  # Train Model
  display(trainer.train())

  # Check performance in validation set
  display(trainer.evaluate())

  # Check how the model fares in our test set.
  display(trainer.predict(test_dataset=tokenized_dataset["test"]))

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/' + model_name)

#### Hyperparameter Tuning

In [14]:
def my_hp_space(trial):
  return {
    "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
    "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [16, 32]),
    "weight_decay": trial.suggest_categorical("weight_decay", [0, 0.01, 0.1]),
    "learning_rate": trial.suggest_categorical("learning_rate", [2e-6, 2e-5, 2e-4]),
    "num_train_epochs": trial.suggest_categorical("num_train_epochs", [3, 5])
  }

def model_init():
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
  model.cuda()

  return model

def my_objective(metrics):
  return metrics["eval_f1"]

def get_trainer_hyper(model_init_, args_, dataset_, tokenizer_, data_collator_, compute_metrics_):
  return Trainer(
      model_init=model_init_,
      args=args_,
      train_dataset=dataset_["train"],
      eval_dataset=dataset_["validation"],
      tokenizer=tokenizer_,
      data_collator=data_collator_,
      compute_metrics=compute_metrics_
  )

def train_model_hyper(model_name):
  global tokenizer
  tokenizer = get_tokenizer(model_name)
  tokenized_dataset = get_tokenized_data(train_valid_test_dataset)

  trainer = get_trainer_hyper(
    model_init,
    get_trainingArgs(),
    tokenized_dataset,
    tokenizer,
    DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics
    )
  
  best_run = trainer.hyperparameter_search(
      hp_space=my_hp_space,
      n_trials=5,
      direction="maximize",
      compute_objective=my_objective
  )

  print("***** Hypertuning best run *****")
  display(best_run)
  
  # Train Model
  for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

  # Save model for future use
  trainer.save_model('/content/drive/Shareddrives/PLN/Assignment 2/models/baseline/hyper/' + model_name)

### PT - Geotrend/distilbert-base-pt-cased

In [ ]:
model_name = "Geotrend/distilbert-base-pt-cased"

In [ ]:
train_model(model_name)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at Geotrend/distilbert-base-pt-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Geotrend/distilbert-base-pt-cased and are newly initialized: ['pre_classifier.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.221100,1.095690,0.534050,0.421932,0.519247,0.398612
2,0.941700,1.080359,0.567503,0.504375,0.574412,0.477762
3,0.786500,1.099895,0.562724,0.515025,0.553936,0.496512


...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5627240143369175,
 'eval_f1': 0.5150253890412397,
 'eval_loss': 1.0998950004577637,
 'eval_precision': 0.5539356475743805,
 'eval_recall': 0.4965120274914089,
 'eval_runtime': 2.2586,
 'eval_samples_per_second': 370.586,
 'eval_steps_per_second': 23.466}

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.3581512 , -0.07337966,  1.7744958 ,  0.3858214 , -2.5542185 ],
       [ 3.2389448 , -1.0849215 ,  0.26063842,  0.43178302, -2.4810872 ],
       [ 2.1245441 , -1.505394  ,  1.7055697 ,  0.6045342 , -2.607756  ],
       ...,
       [ 2.7896938 , -1.7135123 , -0.17353787,  1.7424914 , -2.0651844 ],
       [ 0.62389857, -0.99797255,  2.9277794 , -0.38108924, -1.6441867 ],
       [ 3.0147681 , -1.6656693 ,  0.53686273, -0.12484857, -1.2614144 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/distilbert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/Geotrend/distilbert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81477275f67d8ef9f4af68aeb6799256587efc91da21e00c93a447c4f529447f.6168459fbc8e0022c73def07f9e7976d25cbac85c752238f63549206aeba6abc
Model config DistilBertConfig {
  "_name_or_path": "Geotrend/distilbert-base-pt-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.2",
  "vocab_size": 25000
}

loading file https://huggingface.co/Geotrend/distilbert-base-pt-cased/resolve/main/vocab.txt from cache at /root/.cache/h

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/Geotrend/distilbert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81477275f67d8ef9f4af68aeb6799256587efc91da21e00c93a447c4f529447f.6168459fbc8e0022c73def07f9e7976d25cbac85c752238f63549206aeba6abc
Model config DistilBertConfig {
  "_name_or_path": "Geotrend/distilbert-base-pt-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinuso

...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5101553166069295,
 'eval_f1': 0.5078842181463739,
 'eval_loss': 1.1690025329589844,
 'eval_precision': 0.4754742853078498,
 'eval_recall': 0.5742778050510009,
 'eval_runtime': 2.2849,
 'eval_samples_per_second': 366.313,
 'eval_steps_per_second': 23.195}

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 0.3026111 , -0.95539016,  2.3033242 ,  0.03776403, -2.6459894 ],
       [ 2.0228925 , -1.0478417 ,  0.45456627, -0.04115121, -2.051605  ],
       [ 1.1434606 , -1.4477557 ,  2.0340617 , -0.04298707, -2.3939722 ],
       ...,
       [ 1.4225222 , -1.4597818 , -0.48596388,  1.9393626 , -2.4490993 ],
       [ 0.34697825, -1.5412115 ,  2.9056044 , -0.3209269 , -2.2460303 ],
       [ 1.9788325 , -1.4552298 ,  0.9489511 ,  0.19158314, -2.3184593 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/distilbert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

### PT - Geotrend/bert-base-pt-cased

In [ ]:
model_name = "Geotrend/bert-base-pt-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpv2sqvvbu


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

storing https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/29ca3cacb025b278614e903a0eca5e20b79457c33dcf2fa64250874c94b5a2c7.25d8d06fb0679146a3ed2a3463e3585380bff882fe6e1ebc497196e40dbbd7fa
creating metadata file for /root/.cache/huggingface/transformers/29ca3cacb025b278614e903a0eca5e20b79457c33dcf2fa64250874c94b5a2c7.25d8d06fb0679146a3ed2a3463e3585380bff882fe6e1ebc497196e40dbbd7fa
https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpt8nl5p46


Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

storing https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
creating metadata file for /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
loading configuration file https://huggingface.co/Geotrend/bert-base-pt-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0ebe421d721676677f14dd2556297b76144988966ccb9ca2655253e71be546e1.49ec507c8c81787c05598347db28fe699ccea17f6f63e025aecca400cf4922f8
Model config BertConfig {
  "_name_or_path": "Geotrend/bert-base-pt-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,


...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5770609318996416,
 'eval_f1': 0.5424735695142309,
 'eval_loss': 1.088411808013916,
 'eval_precision': 0.5554884230359712,
 'eval_recall': 0.5388040691648939,
 'eval_runtime': 4.4347,
 'eval_samples_per_second': 188.738,
 'eval_steps_per_second': 11.951}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.8942283 , -1.3650731 ,  1.6932945 ,  0.0049112 , -2.3560195 ],
       [ 3.3103669 , -1.5776519 ,  0.09942892,  1.0624795 , -3.386384  ],
       [ 2.8551533 , -1.4079365 ,  0.980385  , -1.1491941 , -1.5830938 ],
       ...,
       [ 2.1145394 , -2.3779604 ,  2.2831664 ,  0.6204969 , -3.1323304 ],
       [ 0.26888987, -1.8764106 ,  3.569731  ,  0.08218282, -2.3630872 ],
       [ 2.24014   , -2.0484743 ,  2.307457  ,  0.65288764, -3.5304582 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/Geotrend/bert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/158k [00:00<?, ?B/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5495818399044206,
 'eval_f1': 0.553647876541968,
 'eval_loss': 1.1947016716003418,
 'eval_precision': 0.520574755357364,
 'eval_recall': 0.6189347079037801,
 'eval_runtime': 4.7592,
 'eval_samples_per_second': 175.87,
 'eval_steps_per_second': 11.136}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 0.6120045 , -2.1695173 ,  3.9353418 ,  0.25191182, -2.1663191 ],
       [ 2.238734  , -1.8246578 ,  1.3079664 ,  0.5677931 , -2.1862714 ],
       [ 2.1936655 , -1.1421826 ,  1.2952563 , -0.30202937, -1.5177329 ],
       ...,
       [ 1.2099043 , -2.3370407 ,  3.4772475 ,  0.7453207 , -2.5112045 ],
       [ 0.28063256, -1.6676086 ,  4.0433326 ,  0.08551781, -2.2339919 ],
       [ 1.1783977 , -2.1186981 ,  3.5590186 ,  0.60165894, -2.7543602 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/Geotrend/bert-base-pt-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

### PT - neuralmind/bert-base-portuguese-cased


In [15]:
model_name = "neuralmind/bert-base-portuguese-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpal84fxjn


Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f1a9ba41d40e8c6f5ba4988aa2f7702c3b43768183e4b82483e04f2848841ecf.a6c00251b9344c189e2419373d6033016d0cd3d87ea59f6c86069046ac81956d
creating metadata file for /root/.cache/huggingface/transformers/f1a9ba41d40e8c6f5ba4988aa2f7702c3b43768183e4b82483e04f2848841ecf.a6c00251b9344c189e2419373d6033016d0cd3d87ea59f6c86069046ac81956d
https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1boeaqgk


Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
creating metadata file for /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hi

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.6081242532855436,
 'eval_f1': 0.5897821153208468,
 'eval_loss': 0.9645479321479797,
 'eval_precision': 0.6035410312273057,
 'eval_recall': 0.5929285987017947,
 'eval_runtime': 4.7398,
 'eval_samples_per_second': 176.591,
 'eval_steps_per_second': 11.182}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.884908  ,  1.038498  , -2.18935   , -1.6066478 , -0.35243088],
       [ 3.4300473 , -1.994894  ,  0.63945967,  0.72143704, -3.1212835 ],
       [ 3.0750031 , -2.2487645 , -0.12322539, -0.03730537, -1.1943377 ],
       ...,
       [ 2.7760584 , -2.4414334 ,  1.3514451 ,  1.5902003 , -3.3672647 ],
       [ 1.2131273 , -2.2941365 ,  3.2757924 ,  0.8694912 , -2.8195364 ],
       [ 2.1537352 , -2.6220095 ,  2.099932  ,  1.64306   , -3.1421173 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-base-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5818399044205496,
 'eval_f1': 0.588557423152327,
 'eval_loss': 1.0598199367523193,
 'eval_precision': 0.5510828423859349,
 'eval_recall': 0.6632924234986091,
 'eval_runtime': 4.6055,
 'eval_samples_per_second': 181.741,
 'eval_steps_per_second': 11.508}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 0.67140496,  3.7541647 , -2.6057854 , -0.46767405, -1.2402196 ],
       [ 3.176695  , -1.5795118 ,  0.5796741 ,  0.9703557 , -2.4946725 ],
       [ 2.6190624 , -2.5561733 ,  1.1618949 , -0.03252979, -0.86466175],
       ...,
       [ 2.5288785 , -2.5247924 ,  1.4694332 ,  2.1815138 , -2.70129   ],
       [ 0.5530816 , -2.6633098 ,  3.949934  ,  0.7034995 , -2.8342614 ],
       [ 1.4831163 , -2.9828475 ,  2.7009034 ,  2.0874488 , -2.5881813 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-base-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Hyper

In [ ]:
train_model_hyper(model_name)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.084800,0.966433,0.594982,0.542655,0.622347,0.506537
2,0.764800,0.996963,0.597372,0.559421,0.584921,0.556668
3,0.608600,1.092008,0.608124,0.598246,0.584694,0.620953
4,0.500800,1.239289,0.592593,0.582328,0.576989,0.598212
5,0.422300,1.338160,0.579450,0.579239,0.564222,0.601224


...

***** Running Evaluation *****
  Num examples = 837
  Batch size = 32
Saving model checkpoint to ./results/run-4/checkpoint-942
Configuration saved in ./results/run-4/checkpoint-942/config.json
Model weights saved in ./results/run-4/checkpoint-942/pytorch_model.bin
tokenizer config file saved in ./results/run-4/checkpoint-942/tokenizer_config.json
Special tokens file saved in ./results/run-4/checkpoint-942/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 837
  Batch size = 32
Saving model checkpoint to ./results/run-4/checkpoint-1884
Configuration saved in ./results/run-4/checkpoint-1884/config.json
Model weights saved in ./results/run-4/checkpoint-1884/pytorch_model.bin
tokenizer config file saved in ./results/run-4/checkpoint-1884/tokenizer_config.json
Special tokens file saved in ./results/run-4/checkpoint-1884/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 837
  Batch size = 32
Saving model checkpoint to ./results/run-4/checkpoint-282

***** Hypertuning best run *****


BestRun(run_id='0', objective=0.5792388485370905, hyperparameters={'per_device_train_batch_size': 32, 'per_device_eval_batch_size': 32, 'weight_decay': 0.01, 'learning_rate': 2e-05, 'num_train_epochs': 5})

loading configuration file https://huggingface.co/neuralmind/bert-base-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 

Epoch,Training Loss,Validation Loss


In [ ]:
!rm -rf ./results/

### PT - neuralmind/bert-large-portuguese-cased



In [ ]:
model_name = "neuralmind/bert-large-portuguese-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp9ktdi2c8


Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/3a44fa9a74e90f509368a7f2789df38e1fedd153a52c62ef5cc5f4b0f5c99c2a.d61b68f744aef2741575c270d4ba0228cd35693bfa15d8babfb5c1079062d5d7
creating metadata file for /root/.cache/huggingface/transformers/3a44fa9a74e90f509368a7f2789df38e1fedd153a52c62ef5cc5f4b0f5c99c2a.d61b68f744aef2741575c270d4ba0228cd35693bfa15d8babfb5c1079062d5d7
https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_8ua0xd1


Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

storing https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
creating metadata file for /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.6332138590203107,
 'eval_f1': 0.6044611926761922,
 'eval_loss': 0.9485797882080078,
 'eval_precision': 0.6130246798233591,
 'eval_recall': 0.6156894670812196,
 'eval_runtime': 15.4844,
 'eval_samples_per_second': 54.054,
 'eval_steps_per_second': 3.423}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.294826  ,  0.8639877 , -2.1703558 , -0.69185585,  1.223967  ],
       [ 3.4426095 , -2.097659  ,  0.13979225,  1.3672825 , -3.290298  ],
       [ 3.3512785 , -2.2268791 ,  0.9670321 ,  0.431147  , -2.9141436 ],
       ...,
       [ 2.9004357 , -2.9920256 ,  0.9863854 ,  2.2530792 , -3.898819  ],
       [ 1.6162292 , -2.6689003 ,  3.1905425 ,  0.8001343 , -3.0687037 ],
       [ 2.5481427 , -3.2301662 ,  2.228605  ,  1.6023402 , -3.6568592 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/neuralmind/bert-large-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_t

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/neuralmind/bert-large-portuguese-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-large-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5997610513739546,
 'eval_f1': 0.6012315826743313,
 'eval_loss': 0.9024025797843933,
 'eval_precision': 0.5655691371046574,
 'eval_recall': 0.6670632193312607,
 'eval_runtime': 15.429,
 'eval_samples_per_second': 54.248,
 'eval_steps_per_second': 3.435}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 0.6982112 ,  2.392413  , -2.5823116 , -0.51627797,  0.13459474],
       [ 2.2907295 , -0.4781487 , -0.1003031 ,  0.95679337, -2.3349524 ],
       [ 2.24141   , -0.61039203,  0.5383261 ,  0.10189912, -1.9705245 ],
       ...,
       [ 1.9093447 , -1.069753  ,  0.3001502 ,  1.4230311 , -2.3379521 ],
       [ 1.4486129 , -1.7178376 ,  2.529272  ,  0.2485516 , -2.4930284 ],
       [ 1.4877638 , -1.5723784 ,  1.6066962 ,  1.5042164 , -2.8288934 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/neuralmind/bert-large-portuguese-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

### PT - monilouise/ner_news_portuguese 

In [ ]:
train_model("monilouise/ner_news_portuguese")

In [ ]:
!rm -rf ./results/

###  Multi - bert-base-multilingual-cased

In [ ]:
model_name = "bert-base-multilingual-cased"

In [ ]:
train_model(model_name)

https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpis4wvxt4


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjqfnv0he


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidde

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5878136200716846,
 'eval_f1': 0.5579571457945763,
 'eval_loss': 1.0803804397583008,
 'eval_precision': 0.5720279243140933,
 'eval_recall': 0.553404789177985,
 'eval_runtime': 4.538,
 'eval_samples_per_second': 184.444,
 'eval_steps_per_second': 11.679}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 2.0394526 , -1.7366139 ,  2.0997968 ,  0.18916328, -2.4510672 ],
       [ 3.3359878 , -1.5534112 , -0.36108166,  0.33963686, -2.1870446 ],
       [ 2.667591  , -1.8099991 ,  1.8514398 , -0.49706808, -2.4376795 ],
       ...,
       [ 2.8142035 , -1.766973  ,  0.10617247,  1.221561  , -2.9316494 ],
       [ 0.0885511 , -1.8566151 ,  3.6666324 ,  0.13804786, -1.9898982 ],
       [ 2.5178652 , -2.3226469 ,  1.912825  ,  0.7750264 , -3.0698864 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5388291517323776,
 'eval_f1': 0.534668495545153,
 'eval_loss': 1.21450674533844,
 'eval_precision': 0.5017322570229718,
 'eval_recall': 0.602121311296569,
 'eval_runtime': 4.5244,
 'eval_samples_per_second': 184.998,
 'eval_steps_per_second': 11.714}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 0.7706928 , -2.0467553 ,  2.5009148 ,  0.9435435 , -1.8897742 ],
       [ 2.408106  , -0.93433714, -0.06283546,  0.74730736, -2.2646625 ],
       [ 1.7270691 , -1.4986284 ,  2.7311428 , -0.30875364, -2.4046233 ],
       ...,
       [ 0.7469355 , -2.649005  ,  3.1589954 ,  0.7609841 , -2.1072524 ],
       [-0.08701511, -1.8134414 ,  3.6136508 ,  0.14439534, -1.8619947 ],
       [ 2.0092387 , -1.2756716 ,  0.38133833,  1.6718398 , -2.7756975 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-cased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

###  Multi - bert-base-multilingual-uncased

In [ ]:
model_name = "bert-base-multilingual-uncased"

In [ ]:
train_model(model_name)

https://huggingface.co/bert-base-multilingual-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp0z6b930f


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/1b935b135ddb021a7d836c00f5702b80d11d348fd5c5a42cbd933d8ed1f55be9.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/1b935b135ddb021a7d836c00f5702b80d11d348fd5c5a42cbd933d8ed1f55be9.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmps5tz27vj


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
creating metadata file for /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
loading configuration file https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5806451612903226,
 'eval_f1': 0.5497067702204519,
 'eval_loss': 1.0696624517440796,
 'eval_precision': 0.5616965164783988,
 'eval_recall': 0.5472448589974364,
 'eval_runtime': 4.5819,
 'eval_samples_per_second': 182.676,
 'eval_steps_per_second': 11.567}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 1.0068008 ,  1.3165907 , -2.2574716 , -0.64288455,  0.10628735],
       [ 3.5043476 , -1.1374558 , -0.4072238 ,  0.35866255, -2.3819885 ],
       [ 2.4198575 , -1.5340352 ,  2.059636  ,  0.02742954, -2.9252157 ],
       ...,
       [ 1.9868321 , -2.4830842 ,  2.1156018 ,  1.3036941 , -2.6271167 ],
       [ 0.48403916, -1.5527145 ,  3.8177838 , -0.11367209, -2.0803223 ],
       [ 2.7942872 , -1.371459  ,  1.0717577 ,  1.1348625 , -3.3870175 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/bert-base-multilingual-uncased/special_tokens_map.json


In [ ]:
!rm -rf ./results/

#### Custom

In [ ]:
train_model_custom(model_name)

loading configuration file https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": 

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/bert-base-multilingual-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/af4e101d208f361f141144dca21e9c4148aaf0e85441c2e335743d10829c6cad.d63adade93e44e64bedd306ec82ffd33eedabaf0ff08aabe581acaa48616a508
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hid

...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 837
  Batch size = 16


{'epoch': 3.0,
 'eval_accuracy': 0.5507765830346476,
 'eval_f1': 0.5499741037946135,
 'eval_loss': 1.1165486574172974,
 'eval_precision': 0.516997021209161,
 'eval_recall': 0.622147766323024,
 'eval_runtime': 4.5535,
 'eval_samples_per_second': 183.813,
 'eval_steps_per_second': 11.639}

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tokens. If tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 838
  Batch size = 16


PredictionOutput(predictions=array([[ 0.05659453,  2.561615  , -2.585001  , -0.67505676, -0.33670592],
       [ 2.485569  , -1.3612484 ,  1.4399073 ,  0.27740684, -2.1889267 ],
       [ 1.8704886 , -1.2003121 ,  2.6725554 , -0.3221485 , -2.3280303 ],
       ...,
       [ 1.5619184 , -2.2525427 ,  2.4065719 ,  1.1894898 , -1.9993587 ],
       [ 0.24766527, -1.6285247 ,  4.0406485 , -0.12439481, -1.3800536 ],
       [ 1.943152  , -1.284643  ,  1.7819827 ,  1.3548322 , -3.1192431 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 2, 0, 0, 0, 3, 1, 3, 2, 3, 2, 3, 1, 0, 0, 2, 0, 0, 2,
       1, 0, 2, 0, 2, 2, 0, 0, 3, 0, 3, 0, 0, 0, 4, 2, 0, 0, 0, 3, 3, 0,
       0, 2, 0, 3, 3, 2, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 2, 4, 3, 4, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0,
       0, 2, 1, 2, 3, 4, 0, 4, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 3, 0, 0, 0,
       0, 0, 1, 3, 3, 0, 2, 0, 2, 2, 0, 1, 3, 0, 0, 0, 0, 0, 4, 0, 2, 0,
       3, 3, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0

Saving model checkpoint to /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased
Configuration saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/config.json
Model weights saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/PLN/Assignment 2/models/baseline/custom/bert-base-multilingual-uncased/special_tokens_map.json


In [1]:
!rm -rf ./results/